In [1]:
import numpy as np

# Define state 1, 2, 3 ,4 ,5... 19 as normal state with one-hot encoding
# state 0 and state 20 share the same zero feature vectors.

def feature_map(state):
    zero_model = [0]*19
    zero_model[state-1] = 1
    zero_model = np.array(zero_model)
    zero_model.resize((19,1))
    return np.array(zero_model)

# create a hash table to quickly draw features
feature_hash = {0: np.zeros((19,1)),
                20: np.zeros((19,1))}
for state in range(1,20):
    feature_hash[state] = feature_map(state)


In [65]:
history = []
for episode in range(2):
    local = [10]
    state = 10
    while True:
        if np.random.random() > 0.5:
            state += 1
        else:
            state -= 1
        local.append(state)
        if state == 0 or state == 20:
            history.append(local)
            break

In [66]:
# hand pick the hyper parameters
alpha = 0.4
gamma = 0.8
_lambda = 0.9

# set all ones as initialization
w_last_episode = np.ones((19,1))
w_last_round = np.ones((19,1))
w_forward = {}
def n_step_G(t, h, w, hist):
    if h == len(hist):
        # v(T) == 0; reward == 1
        if hist[-1] == 20:
            return gamma**(h-t-1)
        else:
            return 0
    else:
        # reward == 0; 
        return gamma**(h-t)*(w.T@feature_hash[hist[h-1]])
    
def lambda_G(t,h,hist):
    first_term = np.sum([_lambda**(n-1)*n_step_G(t,t+n,w_dict[n-1],hist) for n in range(1, h-t)])
    return (1-_lambda)*first_term + _lambda**(h-t-1)*n_step_G(t,h,w_dict[h-1],hist)
        
for i,hist in enumerate(history):
    w_dict = {0:w_last_episode}
    for h in range(1, len(hist)+1):
        print(f'processing episode {i} horizon {h}', end = '\r')
        w_old = w_last_episode
        for t in range(1,h+1):
            w = w_old + alpha*(lambda_G(t-1,h,hist) - w_old.T@feature_hash[hist[t-1]])*feature_hash[hist[t-1]]
            w_old = w
        else:
            w_dict[h] = w_old
    else:
        w_forward[i] = w_old
        w_last_episode = w_old


In [67]:
w_online = {}
w_2 = np.ones((19,1))
for episode, hist in enumerate(history):
    z = np.zeros((19,1))
    V_old = 0
    for i, state in enumerate(hist):
        if i == len(hist)-2:
            if hist[i+1] == 20:
                R = 1
            else:
                R = 0
            done = True
        else:
            R = 0
            done = False
        V = w_2.T@feature_hash[state]
        V_prime = w_2.T@feature_hash[hist[i+1]]
        delta = R + gamma*V_prime - V
        z = _lambda*gamma*z + (1-alpha*gamma*_lambda*z.T@feature_hash[state])*feature_hash[state]
        w_2 = w_2 + alpha*(delta + V - V_old)*z - alpha*(V-V_old)*feature_hash[state]
        V_old = V_prime
        if done:
            w_online[episode] = w_2
            break

In [70]:
w_forward[0]

array([[1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.53231318],
       [0.32852723],
       [0.27689236],
       [0.28178901],
       [0.2804106 ],
       [0.27976137],
       [0.2788142 ],
       [0.27958496],
       [0.28473191],
       [0.29893919],
       [0.36824546],
       [0.55494604],
       [0.8125568 ]])

In [71]:
w_online[0]

array([[1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.4232541 ],
       [0.15123449],
       [0.02727636],
       [0.01142531],
       [0.00900136],
       [0.01016122],
       [0.01274722],
       [0.01743689],
       [0.04539897],
       [0.12147425],
       [0.28585104],
       [0.58749665],
       [0.90724135]])